In [17]:
# imports
import pandas as pd
import torch
from chess import pgn
import io
import chess
from tqdm.notebook import tqdm

In [18]:
path = '/Users/anu/Documents/GM_games_dataset.csv'

In [19]:
df = pd.read_csv(path, nrows=10000, usecols=['pgn'])


In [20]:
# d = [king, queen, rook, bishop, knight, pawn, black, white, can_move]
def pgn_to_boardrep(game_pgn):
    game = pgn.read_game(game_pgn)
    moves = game.mainline_moves()
    board = chess.Board()
    pieces = chess.PIECE_TYPES
    states_arr = []
    moves_arr = []
    
    for move in moves:
        squares_arr = []
        for square in range(64):
            square_arr = ([int(piece == board.piece_type_at(square)) for piece in pieces] + 
            [int(color == board.color_at(square)) for color in chess.COLORS]) + [int(board.turn == board.color_at(square))]
            squares_arr.append(square_arr)
        
        moves_arr.append([move.from_square, move.to_square])
        
        board.push(move)
        
        states_arr.append(squares_arr)

        
    return (torch.Tensor(states_arr, device='cpu'), torch.Tensor(moves_arr, device='cpu'))

In [21]:
features = torch.Tensor()
games = tqdm(df['pgn'].items(), total=10000)
for game_pgn in games:
    features = torch.cat((features, pgn_to_boardrep(io.StringIO(game_pgn[1]))[0]))

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
features